<a href="https://colab.research.google.com/github/m-adeleke1/Association_of_Data_Scientists/blob/main/RAG_Pinecone_PDF_Reader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing Dependencies

In [1]:
!pip install langchain-openai langchain-community unstructured==0.7.12 pinecone openai tiktoken -q
!pip install langchain langchain-pinecone -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.3/161.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0

## Importing Dependencies

In [2]:
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.vectorstores.pinecone import Pinecone
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain.chains import RetrievalQA
from langchain import OpenAI
from langchain_openai import ChatOpenAI
from langchain_pinecone import PineconeVectorStore
import os
import nltk
from pinecone import Pinecone, ServerlessSpec
import openai
from google.colab import userdata

## Loading URL through LangChain's UnstructuredURLLoader

In [6]:
from langchain.document_loaders import UnstructuredURLLoader
import nltk
nltk.download("punkt_tab")
nltk.download("averaged_perceptron_tagger_eng")
loader = UnstructuredPDFLoader("Amazon Bedrock.pdf")
pdf = loader.load()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [7]:
# Merging urls into a single list
documents = []
documents.extend(pdf)

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
texts = text_splitter.split_documents(documents)
texts

[Document(metadata={'source': 'Amazon Bedrock.pdf'}, page_content="User Guide\n\nAmazon Bedrock\n\nCopyright © 2025 Amazon Web Services, Inc. and/or its aﬃliates. All rights reserved.\n\nAmazon Bedrock\n\nUser Guide\n\nAmazon Bedrock: User Guide\n\nCopyright © 2025 Amazon Web Services, Inc. and/or its aﬃliates. All rights reserved.\n\nAmazon's trademarks and trade dress may not be used in connection with any product or service\n\nthat is not Amazon's, in any manner that is likely to cause confusion among customers, or in any"),
 Document(metadata={'source': 'Amazon Bedrock.pdf'}, page_content='manner that disparages or discredits Amazon. All other trademarks not owned by Amazon are\n\nthe property of their respective owners, who may or may not be aﬃliated with, connected to, or\n\nsponsored by Amazon.\n\nAmazon Bedrock\n\nUser Guide\n\nTable of Contents'),
 Document(metadata={'source': 'Amazon Bedrock.pdf'}, page_content='Table of Contents\n\nWhat is Amazon Bedrock? ...................

## OpenAI API Key Setting



In [9]:
# Set the OpenAI API key as an environment variable
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

## Pinecone API Setting

In [10]:
#Set the pinecone key
pc = Pinecone(api_key = userdata.get("PINECONE_API_KEY"))
os.environ["PINECONE_API_KEY"] = userdata.get("PINECONE_API_KEY")

In [13]:
#Creating a Pinecone index automatically if index doesn't exists
from langchain_pinecone import PineconeVectorStore
embeddings = OpenAIEmbeddings()
index_name = 'myindex' #index name
# if index_name not in pc.list_indexes():
#     # Creating a new index
#     pc.create_index(name=index_name, metric="cosine", dimension=1536,
#                     vector_type="dense",
#                     spec=ServerlessSpec(
#                         cloud="aws",
#                         region="us-east-1"
#                     ))
# The OpenAI embedding model 'text-embedding-ada-002 uses 1536 dimensions'
docsearch = PineconeVectorStore.from_documents(texts, embeddings, index_name=index_name)



## Copilot: URL QA engine using LangChain

In [14]:
llm = ChatOpenAI(temperature=0.1, model_name="gpt-4", max_tokens=256)
qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=docsearch.as_retriever(),
        verbose=True,
        return_source_documents=True)

In [15]:
initial_prompt = "You are a researcher who is going to search the web links, summarize them and share insights as asked"

In [16]:
query = "What is Amazon Titan?"
result = qa({"query": query, "prompt": initial_prompt})

/tmp/ipython-input-2735758286.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({"query": query, "prompt": initial_prompt})




> Entering new RetrievalQA chain...

> Finished chain.


In [17]:
print(result['result'])

Amazon Titan is a family of foundation models pretrained by AWS on large datasets. These are powerful, general-purpose models built to support a variety of use cases. They can be used as-is or privately customized with user's own data. Amazon Titan supports different models for Amazon Bedrock, including Amazon Titan Text, Amazon Titan Text Embeddings V2, Amazon Titan Multimodal Embeddings G1, and Amazon Titan Image Generator G1 V1.


In [18]:
result['source_documents']

[Document(id='38248ebf-200b-45b8-a9a8-c56f15809a9a', metadata={'source': 'Amazon Bedrock.pdf'}, page_content='User Guide\n\nOverview of Amazon Titan models\n\nAmazon Titan foundation models (FMs) are a family of FMs pretrained by AWS on large datasets,\n\nmaking them powerful, general-purpose models built to support a variety of use cases. Use them\n\nas-is or privately customize them with your own data.\n\nAmazon Titan supports the following models for Amazon Bedrock.\n\nAmazon Titan Text\n\nAmazon Titan Text Embeddings V2\n\nAmazon Titan Multimodal Embeddings G1\n\nAmazon Titan Image Generator G1 V1\n\nTopics'),
 Document(id='5586d80b-703e-453d-b882-b9b59d9757a2', metadata={'source': 'Amazon Bedrock.pdf'}, page_content='Amazon Titan models overview ............................................................................................... 2716 Amazon Titan Text ........................................................................................................................